# Loading Neccessary Libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
cd drive/MyDrive/MLasterChef/RecipeBox

/content/drive/.shortcut-targets-by-id/1rWir05GtWo7aaR3D3LOrh_ESmztzjuTj/MLasterChef/RecipeBox


In [6]:
import pandas_access as mdb

val_data_filepath = "/Downloads/val/val_lmdb/data.mdb"

# Listing the tables.
for tbl in mdb.list_tables(val_data_filepath):
    print(tbl)

# Read a small table.
df = pandas_access.read_table(val_data_filepath, "MyTable")

# Read a huge table.
#accumulator = []
#for chunk in pandas_access.read_table("my.mdb", "MyTable", chunksize=10000):
#    accumulator.append(f(chunk))


FileNotFoundError: [Errno 2] No such file or directory: 'mdb-tables'

In [3]:
pip install transformers

     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 6.8 MB 35.8 MB/s 
     |████████████████████████████████| 895 kB 45.7 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 54.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import numpy as np
import pandas as pd
import ast

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer

import copy
import operator
import datetime
import warnings
from itertools import product
warnings.filterwarnings("ignore")

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer



import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt
% matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')



import _pickle as pickle
from os import path

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
import _pickle as pickle
from os import path

# Loading the Data
We will not use `PP_recipies.csv` and `PP_users.csv` but instead will vectorize the data ourselves. The index columns of this dataset is kind of confusing since there are two columns (`u` and `user_id`) for user ids and two columns (`i` and `recipe_id`) for recipe ids.

In [6]:
def load_recipes():
    with open(path.join('data', 'tokens.pkl'), 'rb') as f:
        recipes = pickle.load(f)
    return recipes

r = load_recipes()
r_test = r[1][5].split('; ')
r_test.remove('')

def clean_ing(ing):
    ing_list = ing.split()
    clean = ' '.join(list(filter(lambda x: x.lower() != "advertisement", ing_list)))
    return clean

def clean_ing_list(ing_list):
    ing_list = ing_list.split('; ')
    ing_list.remove('')
    ing_list = [clean_ing(ing) for ing in ing_list if ing != 'advertisement']
    return ing_list

train_data = load_recipes()
ing_data = [clean_ing_list(ing_list) for ing_list in train_data[1]]
numWords = np.array([len(sentence.split()) for sentence in train_data[2]])


recipe_steps = pd.DataFrame()
recipe_steps['ingredients'] = ing_data
recipe_steps['recipe_steps'] = train_data[2]

indexes =np.where(numWords>750)[0]
print(indexes)
recipe_steps = recipe_steps.drop(indexes)

#recipe_steps['recipe_steps'].drop(recipe_steps.index[numWords>750])
#recipe_steps['recipe_steps'][numWords<750]
recipe_steps


[   84   175   566 ... 84897 84913 84931]


,ingredients,recipe_steps
0,"[celery, green pepper finely chopped, finely s...",Toss ingredients lightly and spoon into a butt...
1,"[skirt steak, kosher salt, slices thick cut ap...",Watch how to make this recipe.\nSprinkle the s...
2,"[dried black beans, water, cloves garlic, vege...","In a large saucepan, let the beans soak in eno..."
3,"[ground chuck, one 15-ounce can tomato sauce, ...",Watch how to make this recipe.\nPreheat the ov...
4,"[rice, quinoa, sweet red peppers, oz. white tu...",Special equipment: sushi mat\nCook the brown r...
...,...,...
124468,"[olive oil, minced fresh ginger, cloves garlic...",Heat oil in a large pot over medium heat. Add ...
124469,"[light cream, coconut extract, panang style re...",Heat cream and coconut extract in a skillet or...
124470,"[bacon grease, shredded brussels sprouts, clov...",Heat bacon grease in a skillet over medium-hig...
124471,"[egg yolks, white sugar, eggnog, heavy whippin...",Preheat oven to 350 degrees F (175 degrees C)....


In [7]:
class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    #self.label_ids = []
    self.attn_masks = []

    for index, item in txt_list.iterrows():
      encodings_dict = tokenizer('<|startofing|>'+ '<|ingseparator|>'.join(item['ingredients']) + '<|endofing|>' + '<|startoftext|>'+ item['recipe_steps'] + '<|endoftext|>', truncation=True, max_length=850, padding="max_length")
      #print('<|startofing|>'+ ' <|ingseparator|> '.join(item['ingredients']) + '<|endofing|>')
      #print('<|startoftext|>'+ item['recipe_steps'] + '<|endoftext|>')
      #encodings_labels = tokenizer('<|startoftext|>'+ item['recipe_steps'] + '<|endoftext|>', truncation=True, max_length=750, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      #self.label_ids.append(torch.tensor(encodings_labels['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [8]:
numIngredients = [len(ing) for ing in recipe_steps['ingredients']]
print(max(numIngredients))

78


In [10]:
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium
#tokenizer.add_special_tokens(
#	{'additional_special_tokens': ['<|startofing|>', '<|endofing|>', '<|ingseparator|>']}
#)
tokenizer = GPT2Tokenizer.from_pretrained("./models/tokenizer/Ing")
dataset = GPT2Dataset(recipe_steps, tokenizer, max_length=850)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

batch_size = 2

110,838 training samples
12,316 validation samples


In [11]:
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [12]:
# I'm not really doing anything with the config buheret
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("./models/TrainRecipeBox/Ing")#"gpt2", config=configuration)#('./models/RecipeBoxTrained')#

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [13]:
# some parameters I cooked up that work reasonably well

epochs = 1
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

In [14]:
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [15]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:
            
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()
            model.save_pretrained("./models/TrainRecipeBox/Ing")
            tokenizer.save_pretrained("./models/tokenizer/Ing")

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    model.save_pretrained("./models/TrainRecipeBox/Ing")
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of  55,419. Loss: 0.2473907768726349.   Elapsed: 0:01:59.
0:  bipartisanmilkheavy creamvanilla extractinstant espressowhole wheat flourkosher saltall purpose flourunsweetened cocoa powderfreshly ground nutmegsugarbaking powdersugarcold waterwhole milkunsalted buttereggsone 8-ounce container nonfat evaporated milk (prepared graham crackerspowdered sugarFor the chocolate cupcakes: Preheat the oven to 350 degrees. Whisk together the milk, cream, vanilla, and espresso in a large bowl. Whisk together the flours, the cocoa powder, and the sugar in another bowl. Add the eggs and milk mixture to the milk mixture and mix with your hands until just incorporated. Stir in the flour mixture gently just until just barely incorporated. Pour the batter into a large round tube pan and place it on the center rack of the oven. Bake for 10 to 12 minutes until a wooden pick inserted


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   200  of  55,419. Loss: 0.6186019778251648.   Elapsed: 0:04:08.
0:  increasingbutterground cumincloves garlicground black peppergrated fresh gingerjalapeno peppersred curry pastechili powderMelt butter in skillet over medium heat. Stir in ground cumin and garlic; cook and stir cumin mixture until fragrant, about 30 seconds. Add ginger and jalapeno peppers; cook and stir until fragrant, about 30 seconds. Stir curry paste, chili powder, and chili powder into cumin mixture. Add cooked cilantro; stir until fragrant, about 30 seconds.



...

  Batch 7,100  of  55,419. Loss: 0.22370858490467072.   Elapsed: 2:24:42.
0:  Listenextra virgin olive oilcloves garlicfreshfresh lime juiceworcestershire saucefreshly squeezed lime juicechopped fresh chivesPreheat the oven to 350 degrees F. Heat a large griddle or ovenproof heavy skillet over medium-high heat.
Brush the olive oil on both sides of the garlic cloves and add it to the hot griddle. Stir together to make sure there is no clumps of garlic remaining. Cook, swirling the griddle, just until it begins to bubble and the oil is hot. Sprinkle the garlic butter over the garlic butter and stir. Stir and cook a minute or two until butter is melted and garlic butter is melted. Transfer half of the garlic butter to a medium bowl to cool.
Whisk together the lime juice and Worcestershire sauce and the remaining garlic butter in a medium bowl. Add the lime juice mixture to the skillet and bring to a simmer over medium-low heat.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 7,200  of  55,419. Loss: 0.3315848112106323.   Elapsed: 2:26:44.
0:  dyall purpose flourstick unsalted buttergranulated sugarcocoa powderbaking powdersaltstick unsalted buttervanilla extractmascarpone cheesechopped pecansSpecial equipment: 10-inch metal fondue forks
For the cupcakes: Position an oven rack in the center of the oven and preheat to 350 degrees F. Line a 12-by-18-inch cake pan with 12 cupcake papers.
In a medium-sized mixing bowl, cream together the flour, butter and granulated sugar until smooth, about 3 minutes. Add the cocoa powder, baking powder and salt. Beat until the butter is fully incorporated into the flour mixture. Add the melted butter. Add the vanilla extract.
In a large mixer fitted with the whisk attachment, cream together the butter, sugar, vanilla extract, mascarpone cheese, pecans and chocolate. When the butter is fully incorporated, fold in the flour


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 7,300  of  55,419. Loss: 0.8364984393119812.   Elapsed: 2:28:47.
0:  Domesticgarlic clovesweet redsweet red bell peppersyellow onionschopped fresh flat leaf parsleykosher saltfreshly ground black pepperHeat a large nonstick skillet over medium heat. Add the garlic and stir until golden brown, about 1 minute. Add the chiles, bell peppers and onions. Cook, stirring occasionally, until the onions are tender, about 8 minutes. Stir in the parsley, salt and pepper. Cover and cook, stirring occasionally, until the chili is slightly toasted and the sauce is reduced to about 1/3 cup, about 15 minutes. Serve hot with a small bowl of warm chili topped with a small handful of warm chili.


In [22]:
itera = iter(validation_dataloader)
for i in range(np.random.randint(1, len(validation_dataloader)-1)):
  next(itera)

example = next(itera)[0][0]
ing_end_tok = tokenizer.encode('<|endofing|>')[0]
pad_tok = tokenizer.encode('<|pad|>')

b = example == ing_end_tok
loc = b.nonzero()[0] + 2

input_ex = tokenizer.decode(example[:loc])
input_ex = torch.LongTensor(tokenizer.encode(input_ex))#, padding="max_length", max_length=850))
input_ex = torch.reshape(input_ex, [1, len(input_ex)])

print(input_ex)

tensor([[50259, 26495, 12912,   374,  5126, 20853,   287, 24208, 50261, 48797,
         18873, 13135, 50261,    82, 35652, 50261,  7050, 50261,   501, 34896,
         50261,   293,  2144, 24314,   357, 25968,     8, 50261, 48797,   374,
          5126, 20853,   357, 25968,     8, 50260, 50257]])


In [23]:
input_ex = torch.cat([input_ex, input_ex], 0)
print(input_ex.shape)

torch.Size([2, 37])


In [24]:
model = model.to(device)
s_input_ids = input_ex.to(device)

t0 = time.time()

model.eval()

sample_outputs = model.generate(
    s_input_ids,#bos_token_id=random.randint(1,30000),
    do_sample=True,   
    top_k=50, max_length = 200, top_p=0.95, num_return_sequences=1
                    )

for i, sample_output in enumerate(sample_outputs):
      print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=False)))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: <|startofing|> package frozen raspberries in syrup <|ingseparator|> fresh lemon juice <|ingseparator|> sugar <|ingseparator|> water <|ingseparator|> ice cubes <|ingseparator|> lemon slices (optional) <|ingseparator|> fresh raspberries (optional) <|endofing|> <|startoftext|> In a large bowl, stir together frozen raspberries, raspberries and lemon juice until thoroughly combined. In a blender, blend until smooth. Strain through a fine mesh strainer. Set aside. Chill raspberries for at least 1 hour. In a pitcher with an ice-water scoop, stir in 1 tablespoon sugar and 1 tablespoon water until sugar has dissolved. Add raspberries and lemon slices. Stir in raspberries. Chill thoroughly, stirring occasionally, until thickened, at least 1 hour.
In a pitcher with an ice-water scoop, stir in 1 tablespoon sugar and 1 tablespoon water until sugar has dissolved. Add raspberries and lemon slices. Stir in raspberries. Chill thoroughly, stirring occasionally, until thickened, at least 1 hour.<|endo